<a href="https://colab.research.google.com/github/PedroPertusi/afd-monitor-jogadores/blob/main/data_extractor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Monitor de notícias de jogadores de futebol - Campeonato Brasileiro Série A 2025

Dependências

In [21]:
# !python -m pip install selenium
# !python -m pip install webdriver_manager
# !python -m pip install requests
# !python -m pip install BeautifulSoup4

In [12]:
from bs4 import BeautifulSoup
import requests
import re
import pandas as pd

Função para obter o HTML da página a ser acessada para extração dos dados

In [13]:
def baixar_pagina(url):
  response = requests.get(url)
  response.encoding = "utf-8"
  html = response.text
  return html

Função responsável por obter o link de uma notícia contido em seu respectivo título

In [14]:
def extrair_links_noticias_post(url):
  html = baixar_pagina(url)
  soup = BeautifulSoup(html, 'html.parser')
  title = soup.find_all('h2')
  links_noticias = []
  for t in title:
    if t.find('a').get('href'):
      links_noticias.append(t.find('a').get('href'))
  return links_noticias

Função para extrair o conteúdo de cada notícia

In [15]:
def extrair_conteudo_noticia(url):
    html = baixar_pagina(url)
    soup = BeautifulSoup(html, 'html.parser')
    ps = soup.find_all('p', class_='content-text__container')

    textos = []
    for p in ps:
        # descarta parágrafos que contenham intertítulo ou lista
        if p.find('div', class_='content-intertitle') or p.find('ul'):
            continue
        # extrai texto limpo (inclui <a>, <strong>, etc)
        textos.append(p.get_text(" ", strip=True))
    return textos

Após a definição das funções necessárias para a extração de dados, podemos aplicá-las para realizar a extração

O site escolhido para a obtenção de notícias foi o Globo Esporte (https://ge.globo.com/)

In [16]:
times = ["atletico-mg", "bahia", "botafogo", "bragantino", "ceara", "corinthians", "cruzeiro", "flamengo", "fluminense", "fortaleza",
         "gremio", "internacional", "juventude", "mirassol", "palmeiras", "santos", "sao-paulo", "sport", "vasco", "vitoria"]

conteudos_times = {}

In [17]:
base_url = "https://ge.globo.com/futebol/times/"

for t in times:
  time_url = base_url + t
  noticias = extrair_links_noticias_post(time_url)

  conteudos = []
  for n in noticias:
    conteudos.append(extrair_conteudo_noticia(n))

  pattern = re.compile(r'^\+ O ge')
  c = [" ".join(parts) for parts in conteudos if not pattern.match(" ".join(parts))]

  c = [i for i in c if i not in (" ", "")]

  conteudos_times[t] = c

In [18]:
records = [
    (time, noticia)
    for time, noticias in conteudos_times.items()
    for noticia in noticias
]

df_time_noticia = pd.DataFrame(records, columns=['time', 'noticia'])

Para verificarmos se a extração foi feita corretamente, vamos exibir o DataFrame construído com os times e suas respectivas notícias.

In [19]:
df_time_noticia

,time,noticia
0,atletico-mg,O meia-atacante Dudu esteve na Cidade do Galo ...
1,atletico-mg,O torcedor do Atlético-MG matou a saudade da A...
2,atletico-mg,Os argentinos do Estudiantes de La Plata solta...
3,atletico-mg,"De ""time sem vergonha"" a ""o Galo é o time da v..."
4,atletico-mg,Após a vitória do Atlético-MG contra o Flumine...
...,...,...
75,vitoria,"Após bater o Vasco por 2 a 1 , na noite do últ..."
76,vitoria,O Vitória alcançou o seu principal objetivo pa...
77,vitoria,"Depois de uma longa espera, a estreia de Renat..."
78,vitoria,"Com um jogador a menos, o Vitória mostrou forç..."


Após a validação do DataFrame, podemos salvar estas informações dentro de um arquivo .csv

In [20]:
df_time_noticia.to_csv("times_noticias.csv")